In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np

In [3]:
# !pip install transformers==2.1.0
!pip install gensim==4.1.2
!pip3 install --user underthesea
!pip install tensorflow_addons
# !pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 2.1 MB/s 
     |████████████████████████████████| 581 kB 57.3 MB/s 
     |████████████████████████████████| 965 kB 30.7 MB/s 
     |████████████████████████████████| 235 kB 83.4 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████

In [1]:
#from pyvi import ViTokenizer, ViPosTagger
import regex as re
from underthesea import word_tokenize
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [3]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # tách từ
    document = word_tokenize(document, format="text")
    return document

In [4]:
# lst_stopword = []
# with open('/content/drive/MyDrive/KhanhMD/Data/vietnamese-stopwords.txt','r') as f:
#     for line in f:
#         for word in line.split():
#            lst_stopword.append(word)  
    
# def remove_stopwords(line):
#     words = []
#     for word in line.strip().split():
#         if word not in lst_stopword:
#             words.append(word)
#     return ' '.join(words)

In [5]:
with open('/content/drive/MyDrive/KhanhMD/Data/data_pos.txt', 'r') as file:
    data_pos = file.read().replace('\n', '/27121998/')

In [6]:
with open('/content/drive/MyDrive/KhanhMD/Data/data_neg.txt', 'r') as file:
    data_neg = file.read().replace('\n', '/27121998/')

In [7]:
clean_data_pos = text_preprocess(data_pos)

In [8]:
clean_data_neg = text_preprocess(data_neg)

In [9]:
data_pos[:1000]

'Đây là nhà_thờ được cho là lớn nhất Huế , lại nằm gần_như ở trung_tâm thành_phố . Với lối kiến_trúc của người Pháp , nhà_thờ mang nét cổ_kính pha lẫn với hiện_đại rất châu Âu nên bản_thân mình thấy kiến_trúc nhà_thờ rất đẹp , toát lên cái cao_quý của nhà_thờ . Không_gian ở đây khá yên_bình . Và mỗi dịp giáng_sinh về , nơi đây luôn là nơi rực_rỡ và đẹp nhất .   /27121998/Cơm hến , bún hến ở chợ Phú lộc nghe lâu rồi mà mới có dịp được ăn . Trước giờ toàn ăn cơm hến , bún hến ở đường Hải_Phòng thôi . Có thời_gian lên ăn_ở Chợ_Phú_Lộc , hến béo hơn , thơm hơn nhiều . Công_nhận nước_ngọt nữa . Gía chỉ 10 / tô quá rẻ rồi . /27121998/Resort ngay trung_tâm , cách phố cổ tầm 700m-1km và bạn dễ_dàng đi bộ hoặc đạp xe vào phố . Phòng_ốc xinh_xắn , đặc_biệt là hồ bơi ^ ^ . Hồ bơi không chứa clo mà thay bằng nước_biển lọc mặn . Spa được cộng trong tiền phòng nên mỗi đêm ở bạn được cộng 1 buổi spa luôn nhé /27121998/Chỉ cách phổ cổ khoảng 1km nên khách_sạn này rất tiện_lợi . Do mới xây nên rất mới 

In [10]:
clean_data_neg[:1000]

'mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi hơn 20 hỏi lại thì nv tl có r nhg bảo chờ thêm 15 nữa tụi e lm liền mình k biết có ngon k nhg cũng muốn ăn thử thiết_nghĩ nv quán nên xem_lại cách pv và nc vs khách 27121998 quán này khá là nổi_tiếng nay mới có dịp ghé thử năm ở vii trí khá dễ tìm quán hơi nhỏ đi chiều_tối là đông lắm phải đợi lâu nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm món ăn mình gọi gà sốt phô_mai cay gà quá bở thịt ăn vào k thơm món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy cảm_giác ngán lắm cơm trộn kim_chi hq thì_phải ăn hơi cay nóng_hổi luôn vị ăn ok nhưng k đặc_sắc với giá đó thì ăn k đáng tiền khoai_tây chiên dở và ăn vị k ngon tóm_lại k như kì_vọng của mình giá_cả ở đây quá đắt so với chất_lượng mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt à có nước_lọc miễn_phí nha mọi người tới ăn nhớ hỏi 27121998 thứ 6 nào ta cùng quẩy vuvuzela beer club chung hệ_thống gogi_house daruma kichi kichi nên giá cũng chưa 10 vat thanh_toán trước 20 g

In [11]:
text_pos = clean_data_pos.split(' 27121998 ')

In [12]:
text_pos[:10]

['đây là nhà_thờ được cho là lớn nhất huế lại nằm gần_như ở trung_tâm thành_phố với lối kiến_trúc của người pháp nhà_thờ mang nét cổ_kính pha lẫn với hiện_đại rất châu_âu nên bản_thân mình thấy kiến_trúc nhà_thờ rất đẹp toát lên cái cao_quý của nhà_thờ không_gian ở đây khá yên_bình và mỗi dịp giáng_sinh về nơi đây luôn là nơi rực_rỡ và đẹp nhất',
 'cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà mới có dịp được ăn trước giờ toàn ăn cơm hến bún hến ở đường hải_phòng thôi có thời_gian lên ăn_ở chợ_phú_lộc hến béo hơn thơm hơn nhiều công_nhận nước_ngọt nữa gía chỉ 10 tô quá rẻ rồi',
 'resort ngay trung_tâm cách phố cổ tầm 700 m 1 km và bạn dễ_dàng đi bộ hoặc đạp xe vào phố phòng_ốc xinh_xắn đặc_biệt là hồ bơi hồ bơi không chứa clo mà thay bằng nước_biển lọc mặn spa được cộng trong tiền_phòng nên mỗi đêm ở bạn được cộng 1 buổi spa luôn nhé',
 'chỉ cách phổ cổ khoảng 1 km nên khách_sạn này rất tiện_lợi do mới xây nên rất mới đẹp phòng_ốc có loại phòng nhỏ và phòng bự có bồ sục mình ở loại phò

In [13]:
text_neg = clean_data_neg.split(' 27121998 ')
text_neg[:10]

['mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi hơn 20 hỏi lại thì nv tl có r nhg bảo chờ thêm 15 nữa tụi e lm liền mình k biết có ngon k nhg cũng muốn ăn thử thiết_nghĩ nv quán nên xem_lại cách pv và nc vs khách',
 'quán này khá là nổi_tiếng nay mới có dịp ghé thử năm ở vii trí khá dễ tìm quán hơi nhỏ đi chiều_tối là đông lắm phải đợi lâu nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm món ăn mình gọi gà sốt phô_mai cay gà quá bở thịt ăn vào k thơm món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy cảm_giác ngán lắm cơm trộn kim_chi hq thì_phải ăn hơi cay nóng_hổi luôn vị ăn ok nhưng k đặc_sắc với giá đó thì ăn k đáng tiền khoai_tây chiên dở và ăn vị k ngon tóm_lại k như kì_vọng của mình giá_cả ở đây quá đắt so với chất_lượng mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt à có nước_lọc miễn_phí nha mọi người tới ăn nhớ hỏi',
 'thứ 6 nào ta cùng quẩy vuvuzela beer club chung hệ_thống gogi_house daruma kichi kichi nên giá cũng chưa 10 vat thanh_toán trước 20 g hằng ngà

In [14]:
import pandas as pd
df = pd.DataFrame(text_pos, columns=["text"])
df.to_csv('text_pos.csv', index=False)

In [15]:
text_pos_pd = pd.read_csv('text_pos.csv')
text_pos_pd[:10]

,text
0,đây là nhà_thờ được cho là lớn nhất huế lại nằ...
1,cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà ...
2,resort ngay trung_tâm cách phố cổ tầm 700 m 1 ...
3,chỉ cách phổ cổ khoảng 1 km nên khách_sạn này ...
4,hôm_nay mình quay lại quán cùng_với mấy người ...
5,bánh và nước rất rất ngon phục_vụ nhiệt_tình c...
6,hôm đấy ngẫu_hứng tới nồi vì đang có chương_tr...
7,nghe tên quán_từ rất lâu rồi đọc review cũng t...
8,mình đến nồi_bay hôm trước và có gọi bánh mous...
9,tổng_hợp từ 2 lần em đến với nồi_bay bình_thườ...


In [16]:
text_pos_pd=text_pos_pd.assign(label=lambda x: 1)

In [17]:
df_0 = pd.DataFrame(text_neg, columns=["text"])
df_0.to_csv('text_neg.csv', index=False)

In [18]:
text_neg_pd = pd.read_csv('text_neg.csv')
text_neg_pd[:10]

,text
0,mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi ...
1,quán này khá là nổi_tiếng nay mới có dịp ghé t...
2,thứ 6 nào ta cùng quẩy vuvuzela beer club chun...
3,mình đi với nhóm tổng_cộng 4 người ăn chỉ có k...
4,nhân_viên phục_vụ không mấy tận_tình đồ_ăn ra ...
5,vào đây thì hết bàn nhưng mình vẫn ngồi đợi bì...
6,hôm_nay đến aeon có ghé vào quán món ăn cũng k...
7,mình vào lúc 8 h tối cũng không đông khách lắm...
8,thức_ăn quá_đỗi bình_thường miếng xôi của bạn ...
9,lần đầu_tiên ăn_chắc cũng là lần cuối ăn_ở đây...


In [19]:
text_neg_pd=text_neg_pd.assign(label=lambda x: 0)

In [20]:
df_concat_col = pd.concat([text_pos_pd, text_neg_pd])
df_concat_col.to_csv("data.csv", index=False)

In [21]:
c = pd.read_csv("data.csv")
c[:10]

,text,label
0,đây là nhà_thờ được cho là lớn nhất huế lại nằ...,1
1,cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà ...,1
2,resort ngay trung_tâm cách phố cổ tầm 700 m 1 ...,1
3,chỉ cách phổ cổ khoảng 1 km nên khách_sạn này ...,1
4,hôm_nay mình quay lại quán cùng_với mấy người ...,1
5,bánh và nước rất rất ngon phục_vụ nhiệt_tình c...,1
6,hôm đấy ngẫu_hứng tới nồi vì đang có chương_tr...,1
7,nghe tên quán_từ rất lâu rồi đọc review cũng t...,1
8,mình đến nồi_bay hôm trước và có gọi bánh mous...,1
9,tổng_hợp từ 2 lần em đến với nồi_bay bình_thườ...,1


In [22]:
c = c.sample(frac=1).reset_index(drop=True)

In [23]:
df = c.copy()
df[:50]

,text,label
0,thíc nhất thạch gấu ở đây cực béo và bùi_bùi s...,1
1,hôm_nay mình thử các loại pudding của quán chắ...,0
2,mì hoành_thánh ngon vô_cùng sợi mì dai và chắc...,1
3,không_gian tạm được tầng trệt nhìn chỏi lỏi và...,0
4,quán hơi khó kím một_tẹo không_gian trang_trí ...,0
5,mình mới đi du_lịch ở đây với ông_xã về phải n...,1
6,nướng đá mau chín lắm các b ạ giữ nhiệt tốt tắ...,1
7,mình đi 4 người ăn_no về lết luôn chứ đi không...,1
8,con ở nhà hoài không dám cho ra đường vì đọc b...,1
9,cút chiên ơ đây vẫn là ngon nhất vị vừa thấm g...,1


In [24]:
df.shape

(24734, 2)

In [25]:
text = clean_data_pos +' '+ clean_data_neg

In [26]:
text = text.split(' 27121998 ')

In [27]:
split_text = [s.split() for s in text]

In [28]:
len(split_text)

24734

In [29]:
w2v = gensim.models.Word2Vec(split_text,
                                   vector_size=100,
                                   window=5,
                                   min_count=1)
words = set(w2v.wv.index_to_key)

In [30]:
w2v.train(split_text ,epochs=10,total_examples=len(split_text ))

(17853092, 21409250)

In [31]:
vocab=w2v.wv.key_to_index
print("The total number of words are : ",len(vocab))

The total number of words are :  35482


In [32]:
vocab=list(vocab.keys())

In [33]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v.wv.get_vector(word)
print(len(word_vec_dict))

35482


In [34]:
# maxi=-1
# for i,rev in enumerate(df['text']):
#     tokens=rev.split()
#     if(len(tokens)>maxi):
#         maxi=len(tokens)
# print(maxi)

In [35]:
tok = Tokenizer()
tok.fit_on_texts(df['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['text'])

In [36]:
max_rev_len=125  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100

In [37]:
from keras.preprocessing.sequence import pad_sequences
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape 

(24734, 125)

In [38]:
pad_rev

array([[2447,   97,  538, ...,    0,    0,    0],
       [ 128,  234,    2, ...,    0,    0,    0],
       [ 160, 1170, 1302, ...,    0,    0,    0],
       ...,
       [   7,   99,    2, ...,    0,    0,    0],
       [ 829,  669,  516, ...,  240,  511,  514],
       [ 112,  308,  121, ..., 1671, 2805,  240]], dtype=int32)

In [39]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [40]:
embed_matrix.shape

(18184, 100)

In [41]:
import keras
import keras.utils
from keras import utils as np_utils
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
sklearn.model_selection.train_test_split

<function sklearn.model_selection._split.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)>

In [42]:
# !pip install np_utils
from keras.utils.np_utils import to_categorical

In [43]:
# Y=keras.utils.to_categorical(df['label'])
# Y.shape
from keras import utils as np_utils
Y=keras.utils.np_utils.to_categorical(df['label'])  # one hot target as required by NN.

In [44]:
y = np.array(df['label'])
y = y.reshape(y.size,1)
y

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [45]:

X_train, X_test, y_train, y_test = train_test_split(pad_rev, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.9 x 0.8 = 0.2


In [46]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(SimpleRNN(128))
model.add(Flatten())
model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
model.add(Dropout(0.2))

#model.add(Dense(1024,activation='relu'))
# model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
# model.add(Dropout(0.20))
# model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
# model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
# model.add(Dense(1,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.20))
model.add(Dense(1,activation='sigmoid'))

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 125, 100)          1818400   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               29312     
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,856,033
Trainable params: 1,856,033
Non-

In [48]:
from tensorflow import keras
from keras import optimizers
import tensorflow_addons as tfa

In [49]:
from tensorflow_addons.optimizers import weight_decay_optimizers
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [50]:
epochs=50
batch_size=128

In [51]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [52]:
model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_val,y_val),callbacks=[es, mc])

Epoch 1/50
116/116 [==============================] - ETA: 0s - loss: 1.2734 - accuracy: 0.5280
Epoch 1: val_accuracy improved from -inf to 0.54033, saving model to best_model.h5
116/116 [==============================] - 18s 112ms/step - loss: 1.2734 - accuracy: 0.5280 - val_loss: 1.0614 - val_accuracy: 0.5403
Epoch 2/50
116/116 [==============================] - ETA: 0s - loss: 0.9203 - accuracy: 0.6090
Epoch 2: val_accuracy improved from 0.54033 to 0.65575, saving model to best_model.h5
116/116 [==============================] - 13s 110ms/step - loss: 0.9203 - accuracy: 0.6090 - val_loss: 0.8118 - val_accuracy: 0.6558
Epoch 3/50
116/116 [==============================] - ETA: 0s - loss: 0.7576 - accuracy: 0.6496
Epoch 3: val_accuracy did not improve from 0.65575
116/116 [==============================] - 11s 98ms/step - loss: 0.7576 - accuracy: 0.6496 - val_loss: 0.7243 - val_accuracy: 0.6374
Epoch 4/50
116/116 [==============================] - ETA: 0s - loss: 0.6856 - accuracy: 0.

In [53]:
modelx = keras.models.load_model('best_model.h5')

In [54]:
y_pred_one_hot_encoded = (modelx.predict(X_test)> 0.5).astype("float")
#y_pred_test = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))


In [55]:
print(classification_report(y_test, y_pred_one_hot_encoded,digits=4))

              precision    recall  f1-score   support

           0     0.7885    0.7343    0.7604      2544
           1     0.7378    0.7915    0.7637      2403

    accuracy                         0.7621      4947
   macro avg     0.7631    0.7629    0.7621      4947
weighted avg     0.7639    0.7621    0.7620      4947



In [56]:
modelx.evaluate(X_test, y_test, batch_size=64)

78/78 [==============================] - 1s 11ms/step - loss: 0.5856 - accuracy: 0.7621


[0.5855684280395508, 0.762078046798706]

In [57]:

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [58]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # tách từ
    document = word_tokenize(document, format="text")
    return document

In [59]:
# lst_stopword = []
# with open('/content/drive/MyDrive/KhanhMD/Data/vietnamese-stopwords.txt','r') as f:
#     for line in f:
#         for word in line.split():
#            lst_stopword.append(word)  
    
# def remove_stopwords(line):
#     words = []
#     for word in line.strip().split():
#         if word not in lst_stopword:
#             words.append(word)
#     return ' '.join(words)

In [60]:
document = 'Tụi mình đi 3 đứa , lúc chọn lẩu xog , có nhờ order thêm dùm 2 chai nước với một khay nước_tương , tụi mình nhờ đúng 6 đứa phục_vụ mà nói_đúng_ra tụi nó ừm ờ r kiểu nhường nhau , k có đứa nào lấy hết , phải nói điên tiếc cả người , lúc về kiểu như đuổi khách , làm_ăn như z mất khách hết , một đi KHÔNG_BAO_GIỜ trở_lại'

document = text_preprocess(document)
document = [document]
print(document)
#tok.fit_on_texts(df['text'])
encd = tok.texts_to_sequences(document)
encd

['tụi mình đi 3 đứa lúc chọn lẩu xog có nhờ order thêm dùm 2 chai nước với một khay nước_tương tụi_mình nhờ đúng 6 đứa phục_vụ mà nói_đúng_ra tụi nó ừm ờ r kiểu nhường nhau k có đứa nào lấy hết phải nói điên tiếc cả người lúc về kiểu như đuổi khách làm_ăn như z mất khách hết một đi không_bao_giờ trở_lại']


[[428,
  2,
  23,
  95,
  335,
  89,
  274,
  188,
  1821,
  3,
  846,
  243,
  96,
  1305,
  42,
  718,
  45,
  30,
  84,
  1667,
  45,
  368,
  428,
  2,
  846,
  249,
  439,
  335,
  46,
  38,
  13,
  40,
  249,
  35,
  428,
  247,
  6435,
  2426,
  597,
  235,
  3048,
  357,
  14,
  3,
  335,
  80,
  315,
  88,
  51,
  40,
  1859,
  796,
  52,
  65,
  89,
  86,
  235,
  29,
  1237,
  56,
  78,
  1,
  29,
  2084,
  346,
  56,
  88,
  84,
  23,
  4,
  181,
  166,
  630,
  22]]

In [61]:
pad= pad_sequences(encd, maxlen=max_rev_len, padding='post')

In [62]:
predict_x = modelx.predict(pad)
predict_x

array([[0.12783861]], dtype=float32)